In [1]:
import os
import pandas as pd
import numpy as np
import pymysql
import dataframe_image as dfi

from warnings import filterwarnings
filterwarnings('ignore')

In [49]:
class GetRes:
    def __init__(self, path, host, user, password, database, port):
        self.path = path
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.port=port
        
    def get_cursor(self):
        connection = pymysql.connect(host=self.host, 
                                     user=self.user, 
                                     password=self.password,
                                     database=self.database, 
                                     port=self.port
                                    )
        
        cursor = connection.cursor()
        
        return connection
    
    
    def get_path(self):
        pwd_path = os.getcwd()
        pwd_path = os.path.join(pwd_path, "sqls")
        
        sqls_path = []
        for root, dirs, files in os.walk(pwd_path):
                files = files

        for files in files:
            sql_path = os.path.join(pwd_path, files)
            sqls_path.append(sql_path)
            
#         print(sqls_path)
        return sqls_path                
        
    def get_data(self):
        sqls_path = self.get_path()
        connection = self.get_cursor()
        for sql in sqls_path:
            with open(sql, "r", encoding='UTF-8')as f:
                res = f.read()
#                 print(res)
                    
        data = pd.read_sql(res, connection)
        return data
    
    def data_pivot(self):
        data = self.get_data()
        data_pivot = data.pivot_table(data, index=['f_serveridofevent', 'country', 'first_date'], 
                              columns=['retention_day'],aggfunc={'uid':len})
        data_pivot = data_pivot.droplevel(axis=1, level=0).reset_index()
        
        data_pivot = data_pivot.iloc[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9,16]]
        
        return data_pivot
    
    def data_process(self):
        data = self.get_data()
        
        data_pivot = data.pivot_table(data, index=['f_serveridofevent', 'country', 'first_date'], 
                              columns=['retention_day'],aggfunc={'uid':len})
        data_pivot = data_pivot.droplevel(axis=1, level=0).reset_index()
        
        data_pivot = data_pivot.iloc[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9,16]]
        
        for index in [1, 2, 3, 4, 5, 6,13]:
#           fill 0
            data_pivot["retention_date{}".format(index)] = data_pivot['first_date']+pd.Timedelta(days=index)
            data_pivot["data{}".format(index)] = [0 if i < pd.Timestamp('2022-05-15') else np.nan for i in data_pivot["retention_date{}".format(index)]]
            data_pivot[index].fillna(value=data_pivot["data{}".format(index)], inplace=True)
#           get day 0s users
            data_pivot["0_{}".format(index)]=[0 if pd.isna(i) else np.nan for i in data_pivot[index]]
            data_pivot["0_{}".format(index)].fillna(data_pivot[0], inplace=True)
        
#       groupby 
        data_groupby = data_pivot.groupby(by=['f_serveridofevent', 'country']).sum()
        data_groupby.reset_index(inplace=True)

#           rentention_rate            
        for index in [1, 2, 3, 4, 5, 6,13]:
            data_groupby["day{}".format(index)] = data_groupby[index]/data_groupby["0_{}".format(index)]
        
        data1 = data_groupby.loc[:,['f_serveridofevent', 'country', 0, 'day1', 'day2', 'day3', 'day4', 'day5', 'day6', 'day13']]
        data1.rename(columns={0:'day0'}, inplace=True)
        data1['user_percent'] = data1['day0']/data1.groupby(by=['f_serveridofevent'])['day0'].transform('sum')
        
        data2 = data
        data2['pay_user'] = data2['price'].apply(lambda x: 1 if x>0 else 0)
        data2 = data2.groupby(by=['f_serveridofevent', 'country'])['price', 'pay_user'].sum().reset_index()
        
        data_merge = data2.merge(data1, on=['f_serveridofevent', 'country'], how='left')
        
        data_merge['pay_rate'] = data_merge['pay_user']/data_merge['day0']
        data_merge['arppu'] = data_merge['price']/data_merge['pay_user']
        data_merge['arpu'] = data_merge['pay_rate']*data_merge['arppu']
        
        data_merge.sort_values(by=['f_serveridofevent', 'day0'], ascending=False, inplace=True)
        
        return data_merge
    
    def decorate_data(self):
        data = self.data_process()
        
        format_dict = {'price':"$.{0:.2f}",
               'day0':'{0:.0f}', 'day1':'{0:.2%}', 'day2':'{0:.2%}', 'day3':'{0:.2%}',
               'day4':'{0:.2%}', 'day5':'{0:.2%}', 'day6':'{0:.2%}', 'day13':'{0:.2%}',
               'user_percent':'{0:.2%}', 'pay_rate':'{0:.2%}',
               'arpu':'{0:.2f}', 'arppu':'{0:.2f}'}
        
        serverid = data['f_serveridofevent'].drop_duplicates()
        
        writer = pd.ExcelWriter('test.xlsx')
        
        for i in serverid:
            df = data.loc[data['f_serveridofevent']==i,:].reset_index(drop=True)
            df['temp']=df['day0'].apply(lambda x:0 if x>= 100 else 1)
            
            df = df.style.hide_index()\
                    .format(format_dict, na_rep='-')\
                    .text_gradient(cmap='RdGy', gmap=df['temp'])\
                    .background_gradient(subset=['price'], cmap='Oranges', vmin=100)\
                    .background_gradient(subset=['day1', 'day2', 'day3', 'day4', 'day5', 'day6', 'day13'], cmap='Blues', vmax=0.5)\
                    .bar(subset=['user_percent', 'pay_rate'], color='#E3E1F8', vmax=1)\
                    .hide_columns(['temp'])\
                    .to_excel(writer, sheet_name=i, index=False)
            
            writer.save()
            writer.close()
            
#             dfi.export(df, '3.png', max_rows=-1)        
            
#         return df
            


In [50]:
GetRes = GetRes(path='E:\Github\study_pandas', host='47.243.228.210', 
                    user='dingyanlin', password='dingdingding123', database='fact_20000073_11', port=9030)

In [40]:
raw_data = GetRes.get_data()

In [41]:
raw_data

,date,first_date,retention_day,l_baselevel,country,network,f_serveridofevent,uid,price,total_price,ac_price_tag
0,2022-04-22,2022-04-21,1,8,de,Google Ads ACI,3,102173334000003,0.0,0.00,未付费
1,2022-04-22,2022-04-21,1,8,za,Unattributed,3,103132675000003,0.0,0.00,未付费
2,2022-04-22,2022-04-21,1,1,us,Unattributed,3,106120026000003,0.0,0.00,未付费
3,2022-04-22,2022-04-20,2,7,None,None,3,10891672000003,0.0,0.00,未付费
4,2022-04-22,2022-04-21,1,6,in,Unattributed,3,110313872000003,0.0,0.00,未付费
...,...,...,...,...,...,...,...,...,...,...,...
281894,2022-04-30,2022-04-21,9,15,us,Google Ads ACI,3,66389515000003,0.0,302.63,"(200,500]"
281895,2022-04-30,2022-04-20,10,9,in,Unattributed,3,8462250000003,0.0,0.00,未付费
281896,2022-04-30,2022-04-21,9,6,in,Unattributed,3,88441099000003,0.0,0.00,未付费
281897,2022-04-30,2022-04-21,9,10,in,Unattributed,3,94524104000003,0.0,0.00,未付费


In [42]:
data_pivot = GetRes.data_pivot()

In [43]:
data_pivot

retention_day,f_serveridofevent,country,first_date,0,1,2,3,4,5,6,13
0,3,ad,2022-04-22,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,ad,2022-04-23,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,ae,2022-04-20,54.0,9.0,5.0,3.0,2.0,1.0,1.0,NaN
3,3,ae,2022-04-21,73.0,9.0,6.0,8.0,4.0,4.0,2.0,1.0
4,3,ae,2022-04-22,59.0,10.0,6.0,4.0,6.0,4.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
3829,4,zw,2022-05-05,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN
3830,4,zw,2022-05-07,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3831,4,zw,2022-05-08,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3832,4,zw,2022-05-12,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
data_merge = GetRes.data_process()

In [52]:
data_merge

,f_serveridofevent,country,price,pay_user,day0,day1,day2,day3,day4,day5,day6,day13,user_percent,pay_rate,arppu,arpu
285,4,in,449.84,248,39503.0,0.189625,0.106982,0.080721,0.063205,0.053207,0.047479,0.029417,0.452357,0.006278,1.813871,0.011387
387,4,us,10723.04,793,6278.0,0.249229,0.159550,0.127312,0.111416,0.099720,0.095264,0.045097,0.071891,0.126314,13.522119,1.708034
247,4,de,2448.71,229,4291.0,0.161699,0.096851,0.068480,0.055649,0.046833,0.046782,0.048924,0.049137,0.053368,10.693057,0.570662
262,4,fr,354.04,61,3374.0,0.145118,0.074609,0.059571,0.040454,0.034631,0.028793,0.024883,0.038636,0.018079,5.803934,0.104932
345,4,pk,16.87,12,3113.0,0.168780,0.094203,0.078543,0.068942,0.059065,0.052189,0.027363,0.035648,0.003855,1.405833,0.005419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,3,td,0.00,0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000017,0.000000,NaN,NaN
181,3,to,0.00,0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000017,0.000000,NaN,NaN
193,3,vi,0.00,0,1.0,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000017,0.000000,NaN,NaN
195,3,vu,0.00,0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000017,0.000000,NaN,NaN


In [53]:
GetRes.decorate_data()

In [78]:
df = data_merge
df

,f_serveridofevent,country,price,pay_user,day0,day1,day2,day3,day4,day5,day6,day13,user_percent,pay_rate,arppu,arpu
285,4,in,449.84,248,39503.0,0.189625,0.106982,0.080721,0.063205,0.053207,0.047479,0.029417,0.452357,0.006278,1.813871,0.011387
387,4,us,10723.04,793,6278.0,0.249229,0.159550,0.127312,0.111416,0.099720,0.095264,0.045097,0.071891,0.126314,13.522119,1.708034
247,4,de,2448.71,229,4291.0,0.161699,0.096851,0.068480,0.055649,0.046833,0.046782,0.048924,0.049137,0.053368,10.693057,0.570662
262,4,fr,354.04,61,3374.0,0.145118,0.074609,0.059571,0.040454,0.034631,0.028793,0.024883,0.038636,0.018079,5.803934,0.104932
345,4,pk,16.87,12,3113.0,0.168780,0.094203,0.078543,0.068942,0.059065,0.052189,0.027363,0.035648,0.003855,1.405833,0.005419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,3,td,0.00,0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000017,0.000000,NaN,NaN
181,3,to,0.00,0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000017,0.000000,NaN,NaN
193,3,vi,0.00,0,1.0,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000017,0.000000,NaN,NaN
195,3,vu,0.00,0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000017,0.000000,NaN,NaN


In [63]:
for index in [1, 2, 3, 4, 5, 6,13]:    
    df['day{}'.format(index)] = df['day{}'.format(index)].apply(lambda x: format(x, '.2%'))
df

,f_serveridofevent,country,price,pay_user,day0,day1,day2,day3,day4,day5,day6,day13,user_percent,pay_rate,arppu,arpu
285,4,in,449.84,248,39503.0,18.96%,10.70%,8.07%,6.32%,5.32%,4.75%,2.94%,0.452357,0.006278,1.813871,0.011387
387,4,us,10723.04,793,6278.0,24.92%,15.95%,12.73%,11.14%,9.97%,9.53%,4.51%,0.071891,0.126314,13.522119,1.708034
247,4,de,2448.71,229,4291.0,16.17%,9.69%,6.85%,5.56%,4.68%,4.68%,4.89%,0.049137,0.053368,10.693057,0.570662
262,4,fr,354.04,61,3374.0,14.51%,7.46%,5.96%,4.05%,3.46%,2.88%,2.49%,0.038636,0.018079,5.803934,0.104932
345,4,pk,16.87,12,3113.0,16.88%,9.42%,7.85%,6.89%,5.91%,5.22%,2.74%,0.035648,0.003855,1.405833,0.005419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,3,td,0.00,0,1.0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.000017,0.000000,NaN,NaN
181,3,to,0.00,0,1.0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.000017,0.000000,NaN,NaN
193,3,vi,0.00,0,1.0,100.00%,0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.000017,0.000000,NaN,NaN
195,3,vu,0.00,0,1.0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.000017,0.000000,NaN,NaN


In [93]:
def color_returns(val):
    if val > 0.3:
        color = '#f7797d'
    
    if val > 0.15:
        color = '#fbd786'
        
    else:
        color = '#c6ffdd'
    
    return f'background-color:{color}'


format_dict = {'price':"$.{0:.2f}",
       'day0':'{0:.0f}', 'day1':'{0:.2%}', 'day2':'{0:.2%}', 'day3':'{0:.2%}',
       'day4':'{0:.2%}', 'day5':'{0:.2%}', 'day6':'{0:.2%}', 'day13':'{0:.2%}',
       'user_percent':'{0:.2%}', 'pay_rate':'{0:.2%}',
       'arpu':'{0:.2f}', 'arppu':'{0:.2f}'}

df.style.hide_index()\
        .format(format_dict, na_rep='-')\
        .applymap(color_returns, subset=['day1', 'day2', 'day3', 'day4', 'day5', 'day6', 'day13'])

f_serveridofevent,country,price,pay_user,day0,day1,day2,day3,day4,day5,day6,day13,user_percent,pay_rate,arppu,arpu
4,in,$.449.84,248,39503,18.96%,10.70%,8.07%,6.32%,5.32%,4.75%,2.94%,45.24%,0.63%,1.81,0.01
4,us,$.10723.04,793,6278,24.92%,15.95%,12.73%,11.14%,9.97%,9.53%,4.51%,7.19%,12.63%,13.52,1.71
4,de,$.2448.71,229,4291,16.17%,9.69%,6.85%,5.56%,4.68%,4.68%,4.89%,4.91%,5.34%,10.69,0.57
4,fr,$.354.04,61,3374,14.51%,7.46%,5.96%,4.05%,3.46%,2.88%,2.49%,3.86%,1.81%,5.80,0.10
4,pk,$.16.87,12,3113,16.88%,9.42%,7.85%,6.89%,5.91%,5.22%,2.74%,3.56%,0.39%,1.41,0.01
4,ae,$.239.59,15,2817,9.90%,5.23%,3.79%,3.11%,2.66%,1.73%,1.32%,3.23%,0.53%,15.97,0.09
4,it,$.31.80,14,1991,10.25%,5.86%,4.40%,3.64%,2.92%,2.38%,0.58%,2.28%,0.70%,2.27,0.02
4,np,$.0.99,1,1918,14.83%,6.89%,5.06%,3.08%,3.29%,2.14%,1.10%,2.20%,0.05%,0.99,0.00
4,gb,$.548.28,104,1889,17.25%,10.06%,6.71%,5.83%,4.76%,4.12%,2.74%,2.16%,5.51%,5.27,0.29
4,sa,$.21.86,11,1604,11.71%,6.14%,3.92%,3.22%,2.12%,1.86%,0.80%,1.84%,0.69%,1.99,0.01


In [64]:
df

,f_serveridofevent,country,price,pay_user,day0,day1,day2,day3,day4,day5,day6,day13,user_percent,pay_rate,arppu,arpu
285,4,in,449.84,248,39503.0,18.96%,10.70%,8.07%,6.32%,5.32%,4.75%,2.94%,0.452357,0.006278,1.813871,0.011387
387,4,us,10723.04,793,6278.0,24.92%,15.95%,12.73%,11.14%,9.97%,9.53%,4.51%,0.071891,0.126314,13.522119,1.708034
247,4,de,2448.71,229,4291.0,16.17%,9.69%,6.85%,5.56%,4.68%,4.68%,4.89%,0.049137,0.053368,10.693057,0.570662
262,4,fr,354.04,61,3374.0,14.51%,7.46%,5.96%,4.05%,3.46%,2.88%,2.49%,0.038636,0.018079,5.803934,0.104932
345,4,pk,16.87,12,3113.0,16.88%,9.42%,7.85%,6.89%,5.91%,5.22%,2.74%,0.035648,0.003855,1.405833,0.005419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,3,td,0.00,0,1.0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.000017,0.000000,NaN,NaN
181,3,to,0.00,0,1.0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.000017,0.000000,NaN,NaN
193,3,vi,0.00,0,1.0,100.00%,0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.000017,0.000000,NaN,NaN
195,3,vu,0.00,0,1.0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.000017,0.000000,NaN,NaN


In [90]:
df.style.hide_index()\
        .format(format_dict, na_rep='-')\
        .applymap(color_returns, subset=['day1', 'day2', 'day3', 'day4', 'day5', 'day6', 'day13'])


f_serveridofevent,country,price,pay_user,day0,day1,day2,day3,day4,day5,day6,day13,user_percent,pay_rate,arppu,arpu
4,in,$.449.84,248,39503,18.96%,10.70%,8.07%,6.32%,5.32%,4.75%,2.94%,45.24%,0.63%,1.81,0.01
4,us,$.10723.04,793,6278,24.92%,15.95%,12.73%,11.14%,9.97%,9.53%,4.51%,7.19%,12.63%,13.52,1.71
4,de,$.2448.71,229,4291,16.17%,9.69%,6.85%,5.56%,4.68%,4.68%,4.89%,4.91%,5.34%,10.69,0.57
4,fr,$.354.04,61,3374,14.51%,7.46%,5.96%,4.05%,3.46%,2.88%,2.49%,3.86%,1.81%,5.80,0.10
4,pk,$.16.87,12,3113,16.88%,9.42%,7.85%,6.89%,5.91%,5.22%,2.74%,3.56%,0.39%,1.41,0.01
4,ae,$.239.59,15,2817,9.90%,5.23%,3.79%,3.11%,2.66%,1.73%,1.32%,3.23%,0.53%,15.97,0.09
4,it,$.31.80,14,1991,10.25%,5.86%,4.40%,3.64%,2.92%,2.38%,0.58%,2.28%,0.70%,2.27,0.02
4,np,$.0.99,1,1918,14.83%,6.89%,5.06%,3.08%,3.29%,2.14%,1.10%,2.20%,0.05%,0.99,0.00
4,gb,$.548.28,104,1889,17.25%,10.06%,6.71%,5.83%,4.76%,4.12%,2.74%,2.16%,5.51%,5.27,0.29
4,sa,$.21.86,11,1604,11.71%,6.14%,3.92%,3.22%,2.12%,1.86%,0.80%,1.84%,0.69%,1.99,0.01


In [82]:
color_returns(0.2)

'backgroud-color:#3fc1c9'